In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from datetime import datetime

### Cluster Configuration <br>
Driver Memory: 30.5GB <br>
Worker Memory: 30.5GB <br>
Worker cores: 4 <br>
Minimum number of workers: 2 <br>
Maximum number of workers: 8 <br>

In [0]:
spark = SparkSession.builder \
            .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.1")\
            .appName("project") \
            .getOrCreate()

In [0]:
aws_access_key = '***'
aws_secret_key = '***'
spark._jsc.hadoopConfiguration().set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.3.1')
#spark._jsc.hadoopConfiguration().set('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector_2.12:3.0.1') 
spark._jsc.hadoopConfiguration().set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
spark._jsc.hadoopConfiguration().set("fs.s3a.access.key", aws_access_key)
spark._jsc.hadoopConfiguration().set("fs.s3a.secret.key", aws_secret_key)

In [0]:
spark._jsc.hadoopConfiguration().set('spark.network.timeout','7200s')
spark._jsc.hadoopConfiguration().set('spark.executor.heartbeatInterval','1200s')

#### DataType cast functions

In [0]:
def IntegerSafe(value): # In case there are non-integer type to be converted.
    try:
        return int(value)
    except:
        return None

In [0]:
def TimeStampSafe(value):
    value = value.strip() # Timestamp starting and ending with a double quotation mark.
    try:
        value = int(value)
        return datetime.utcfromtimestamp(value)
    except ValueError:
        return None


In [0]:
def YearSafe(value):
    value = value.strip() # Timestamp starting and ending with a double quotation mark.
    try:
        return datetime.strptime(value, "%Y")
    except ValueError:
        return None

#### Load recommendation log histroy file

In [0]:
rec_log_schema = StructType([StructField("UserId", IntegerType(), False),\
    StructField("ItemId", IntegerType(), False),\
    StructField("Result", IntegerType(), False),
    StructField("Unix-timestamp", TimestampType(), True)])

In [0]:
%%time
rec_log_rdd = sc.textFile('s3://msds697-group18-2022/track1/rec_log_train.txt')\
                .map(lambda x: x.split('\t'))\
                .map(lambda x : (IntegerSafe(x[0]), IntegerSafe(x[1]), IntegerSafe(x[2]), TimeStampSafe(x[3])))

CPU times: user 1.26 ms, sys: 0 ns, total: 1.26 ms
Wall time: 303 ms

In [0]:
rec_log_rdd.first()

Out[10]: (2088948, 1760350, -1, datetime.datetime(2011, 10, 11, 15, 59, 45))

In [0]:
%%time
rec_log_df = spark.createDataFrame(rec_log_rdd, rec_log_schema)
rec_log_df.printSchema()

root
-- UserId: integer (nullable = false)
-- ItemId: integer (nullable = false)
-- Result: integer (nullable = false)
-- Unix-timestamp: timestamp (nullable = true)

CPU times: user 194 ms, sys: 32.3 ms, total: 227 ms
Wall time: 359 ms

In [0]:
%%time
rec_log_df.show()

+-------+-------+------+-------------------+
 UserId| ItemId|Result| Unix-timestamp|
+-------+-------+------+-------------------+
2088948|1760350| -1|2011-10-11 15:59:45|
2088948|1774722| -1|2011-10-11 15:59:45|
2088948| 786313| -1|2011-10-11 15:59:45|
 601635|1775029| -1|2011-10-11 15:59:45|
 601635|1902321| -1|2011-10-11 15:59:45|
 601635| 462104| -1|2011-10-11 15:59:45|
1529353|1774509| -1|2011-10-11 15:59:46|
1529353|1774717| -1|2011-10-11 15:59:46|
1529353|1775024| -1|2011-10-11 15:59:46|
1853982|1760403| -1|2011-10-11 15:59:49|
1853982|2105482| -1|2011-10-11 15:59:49|
1853982| 725387| -1|2011-10-11 15:59:49|
2202935|1774595| -1|2011-10-11 15:59:49|
2202935|1774796| -1|2011-10-11 15:59:49|
2202935| 458026| -1|2011-10-11 15:59:49|
 451392|1774509| -1|2011-10-11 15:59:50|
 451392|1774713| -1|2011-10-11 15:59:50|
 451392| 647356| -1|2011-10-11 15:59:50|
2270362|1774722| -1|2011-10-11 15:59:51|
2270362| 563514| -1|2011-10-11 15:59:51|
+-------+-------+------+-------------------+
only showing top 20 rows

CPU times: user 5.89 ms, sys: 0 ns, total: 5.89 ms
Wall time: 2.2 s

#### Load user profile data

In [0]:
user_profile_schema = StructType([StructField("UserId", IntegerType(), False),\
    StructField("Year_of_birth", IntegerType(), True),\
    StructField("Gender", IntegerType(), False),
    StructField("Number_of_tweets", IntegerType(), False),
    StructField("TagIds", StringType(), True)])

In [0]:
%%time
user_profile_rdd = sc.textFile('s3://msds697-group18-2022/track1/user_profile.txt')\
                .map(lambda x: x.split('\t'))\
                .map(lambda x : (IntegerSafe(x[0]), IntegerSafe(x[1]), IntegerSafe(x[2]), IntegerSafe(x[3]), x[4]))

CPU times: user 912 µs, sys: 115 µs, total: 1.03 ms
Wall time: 40.8 ms

In [0]:
%%time
user_profile_rdd.first()

CPU times: user 1.75 ms, sys: 3.96 ms, total: 5.72 ms
Wall time: 320 ms
Out[29]: (100044, 1899, 1, 5, '831;55;198;8;450;7;39;5;111')

In [0]:
%%time
user_profile_df = spark.createDataFrame(user_profile_rdd, user_profile_schema)
user_profile_df.printSchema()

root
-- UserId: integer (nullable = false)
-- Year_of_birth: integer (nullable = true)
-- Gender: integer (nullable = false)
-- Number_of_tweets: integer (nullable = false)
-- TagIds: string (nullable = true)

CPU times: user 1.26 ms, sys: 8.71 ms, total: 9.96 ms
Wall time: 49.6 ms

In [0]:
%%time
user_profile_df.show()

+------+-------------+------+----------------+--------------------+
UserId|Year_of_birth|Gender|Number_of_tweets| TagIds|
+------+-------------+------+----------------+--------------------+
100044| 1899| 1| 5|831;55;198;8;450;...|
100054| 1987| 2| 6| 0|
100065| 1989| 1| 57| 0|
100080| 1986| 1| 31|113;41;44;48;91;9...|
100086| 1986| 1| 129| 0|
100097| 1981| 1| 75| 0|
100100| 1984| 1| 47| 71;51|
100101| 2003| 1| 31| 0|
100116| 1988| 2| 39| 35|
100117| 2009| 2| 37| 0|
100122| 1891| 1| 17| 0|
100133| 2004| 2| 3| 0|
100136| 1991| 1| 345| 0|
100137| 2011| 1| 44| 417;63;13;2|
100144| 1987| 1| 39| 0|
100153| 1990| 1| 31|1;4;100;18;71;64;...|
100155| 1986| 1| 32| 0|
100160| null| 0| 6| 0|
100177| 1983| 1| 70|120;8;48;19;86;49...|
100187| 1984| 2| 222|88;109;46;14332;6...|
+------+-------------+------+----------------+--------------------+
only showing top 20 rows

CPU times: user 446 µs, sys: 58 µs, total: 504 µs
Wall time: 395 ms

#### Convert String type Tag Ids into a list of Tag Ids

In [0]:
def split_Ids(tagids):
    
    '''
    convert string tagIDs in format "831;55;198;8;450" 
    to Array of integer elements --> [831,55,198,8,450]
    '''
    
    if tagids == '0':
        return []
    else:
        return [ IntegerSafe(x) for x in tagids.split(';') ]   

In [0]:
# Register function for unwrapping of Tagids into Array as UDF
clean_ids = udf(split_Ids, ArrayType(IntegerType()))

In [0]:
%%time
user_profile_df = user_profile_df.withColumn('Tag_Ids_list', clean_ids('TagIds'))\
                                 .drop('TagIds')

CPU times: user 4.85 ms, sys: 638 µs, total: 5.49 ms
Wall time: 44.6 ms

In [0]:
%%time
user_details = rec_log_df.join(user_profile_df, 'UserId', 'left_outer')

CPU times: user 0 ns, sys: 950 µs, total: 950 µs
Wall time: 12.5 ms

In [0]:
%%time
user_details.show()

+------+-------+------+-------------------+-------------+------+----------------+--------------------+
UserId| ItemId|Result| Unix-timestamp|Year_of_birth|Gender|Number_of_tweets| Tag_Ids_list|
+------+-------+------+-------------------+-------------+------+----------------+--------------------+
114897|1760377| -1|2011-10-12 15:29:22| 1974| 2| 1892| []|
114897|1932882| -1|2011-10-12 15:29:22| 1974| 2| 1892| []|
114897| 713228| -1|2011-10-12 15:29:22| 1974| 2| 1892| []|
244315|1213266| -1|2011-10-29 12:57:52| 1995| 2| 68| []|
283440|1606902| -1|2011-11-05 07:28:22| 1989| 2| 14|[30, 1, 8, 40, 34...|
495557|1494039| -1|2011-11-11 02:18:10| 1990| 2| 2| []|
495557|1606717| -1|2011-11-11 02:18:10| 1990| 2| 2| []|
495557|1774717| -1|2011-11-11 02:18:10| 1990| 2| 2| []|
601635|1775029| -1|2011-10-11 15:59:45| 1996| 2| 701| [16, 17, 20, 28]|
601635|1902321| -1|2011-10-11 15:59:45| 1996| 2| 701| [16, 17, 20, 28]|
601635| 462104| -1|2011-10-11 15:59:45| 1996| 2| 701| [16, 17, 20, 28]|
638062|1606902| -1|2011-10-28 01:02:32| 1986| 2| 480| [94, 1, 68, 20, 8]|
638062| 516024| -1|2011-10-28 01:02:32| 1986| 2| 480| [94, 1, 68, 20, 8]|
725911|1760401| -1|2011-10-24 13:09:54| 1899| 2| 54| []|
725911|1774457| -1|2011-10-24 13:09:54| 1899| 2| 54| []|
725911| 731434| -1|2011-10-24 13:09:54| 1899| 2| 54| []|
762757|1606515| -1|2011-11-02 10:23:06| 1977| 1| 62|[28170, 8484, 628...|
762757|1606596| -1|2011-11-02 10:23:06| 1977| 1| 62|[28170, 8484, 628...|
762757|1606902| -1|2011-11-02 10:23:06| 1977| 1| 62|[28170, 8484, 628...|
762847|1837210| -1|2011-10-31 09:35:11| 2009| 1| 0| []|
+------+-------+------+-------------------+-------------+------+----------------+--------------------+
only showing top 20 rows

CPU times: user 45.6 ms, sys: 39.8 ms, total: 85.4 ms
Wall time: 21.3 s

#### Load item features

In [0]:
item_schema = StructType([StructField("ItemId", IntegerType(), False),\
    StructField("Category", StringType(), True),\
    StructField("Keywords", StringType(), False)])

In [0]:
%%time
item_rdd = sc.textFile('s3://msds697-group18-2022/track1/item.txt')\
                .map(lambda x: x.split('\t'))\
                .map(lambda x : (IntegerSafe(x[0]), x[1], x[2]))

CPU times: user 913 µs, sys: 130 µs, total: 1.04 ms
Wall time: 29.6 ms

In [0]:
item_rdd.first()

Out[39]: (2335869,
 '8.1.4.2',
 '412042;974;85658;174033;974;9525;72246;39928;8895;30066;2245;1670;85658;174033;6977;6183;974;85658;174033;974;9525;72246;39928;8895;30066;2245;1670;85658;174033;6977;6183;974')

In [0]:
%%time
item_df = spark.createDataFrame(item_rdd, item_schema)
item_df.printSchema()

root
-- ItemId: integer (nullable = false)
-- Category: string (nullable = true)
-- Keywords: string (nullable = false)

CPU times: user 1.94 ms, sys: 3.97 ms, total: 5.91 ms
Wall time: 42.1 ms

In [0]:
%%time
item_df.show()

+-------+--------+--------------------+
 ItemId|Category| Keywords|
+-------+--------+--------------------+
2335869| 8.1.4.2|412042;974;85658;...|
1774844| 1.8.3.6|31449;517124;4500...|
1775000| 1.4.2.4|259580;626835;121...|
1775024| 1.4.1.4|498354;61029;6077...|
1774455| 1.4.1.2|155009;345081;122...|
1775040| 1.4.2.2|100947;97714;1220...|
1774505| 1.4.9.2|254337;195099;974...|
1774776| 8.1.4.2|239661;974;46479;...|
 495072| 8.1.4.2|296259;596521;486...|
2076876| 1.4.9.2|239661;974;428257...|
 163267| 8.1.4.2|239661;588100;571...|
 218438| 1.4.9.2|974;97714;239661;...|
1775005| 1.4.1.1|228377;573290;122...|
1932882| 1.4.1.2|209854;555490;122...|
 303060| 1.4.2.2|606496;81309;1220...|
1774456| 1.4.2.1|3203;19818;12203;...|
 709795| 1.9.2.5|541405;677101;122...|
1775038| 1.4.1.2|155009;7931;12203...|
1774457| 1.4.1.2|712604;209854;122...|
 504783| 1.4.1.1|589692;12203;1231...|
+-------+--------+--------------------+
only showing top 20 rows

CPU times: user 4.59 ms, sys: 665 µs, total: 5.26 ms
Wall time: 279 ms

#### Convert String type Keywords into a list of keywords

In [0]:
%%time
item_df = item_df.withColumn('Keywords_list', clean_ids('Keywords'))\
                                 .drop('Keywords')

CPU times: user 1.27 ms, sys: 185 µs, total: 1.45 ms
Wall time: 20.1 ms

In [0]:
%%time
log_details = user_details.join(item_df, 'ItemId', 'left_outer')

CPU times: user 733 µs, sys: 107 µs, total: 840 µs
Wall time: 15.2 ms

In [0]:
%%time
log_details.show()

+-------+------+------+-------------------+-------------+------+----------------+--------------------+--------+--------------------+
 ItemId|UserId|Result| Unix-timestamp|Year_of_birth|Gender|Number_of_tweets| Tag_Ids_list|Category| Keywords_list|
+-------+------+------+-------------------+-------------+------+----------------+--------------------+--------+--------------------+
1760377|114897| -1|2011-10-12 15:29:22| 1974| 2| 1892| []| 1.4.1.5|[20819, 396947, 7...|
1932882|114897| -1|2011-10-12 15:29:22| 1974| 2| 1892| []| 1.4.1.2|[209854, 555490, ...|
 713228|114897| -1|2011-10-12 15:29:22| 1974| 2| 1892| []|1.4.1.13|[650, 517117, 517...|
1213266|244315| -1|2011-10-29 12:57:52| 1995| 2| 68| []| 1.1.1.2|[60705, 449826, 2...|
1606902|283440| -1|2011-11-05 07:28:22| 1989| 2| 14|[30, 1, 8, 40, 34...| 1.1.2.1|[619870, 242039, ...|
1494039|495557| -1|2011-11-11 02:18:10| 1990| 2| 2| []| 1.1.2.2|[38813, 674057, 3...|
1606717|495557| -1|2011-11-11 02:18:10| 1990| 2| 2| []| 8.2.5.2|[6359, 12641, 232...|
1774717|495557| -1|2011-11-11 02:18:10| 1990| 2| 2| []| 1.4.9.2|[1202, 974, 25433...|
1775029|601635| -1|2011-10-11 15:59:45| 1996| 2| 701| [16, 17, 20, 28]| 1.4.9.2|[575248, 629557, ...|
1902321|601635| -1|2011-10-11 15:59:45| 1996| 2| 701| [16, 17, 20, 28]|1.11.2.4|[24357, 118825, 2...|
 462104|601635| -1|2011-10-11 15:59:45| 1996| 2| 701| [16, 17, 20, 28]| 1.5.3.7|[30630, 15471, 90...|
1606902|638062| -1|2011-10-28 01:02:32| 1986| 2| 480| [94, 1, 68, 20, 8]| 1.1.2.1|[619870, 242039, ...|
 516024|638062| -1|2011-10-28 01:02:32| 1986| 2| 480| [94, 1, 68, 20, 8]| 1.9.2.1|[528997, 18059, 7...|
1760401|725911| -1|2011-10-24 13:09:54| 1899| 2| 54| []| 1.4.1.5|[37009, 582807, 5...|
1774457|725911| -1|2011-10-24 13:09:54| 1899| 2| 54| []| 1.4.1.2|[712604, 209854, ...|
 731434|725911| -1|2011-10-24 13:09:54| 1899| 2| 54| []| 1.4.9.2|[632914, 974, 618...|
1606515|762757| -1|2011-11-02 10:23:06| 1977| 1| 62|[28170, 8484, 628...| 8.1.1.2|[6359, 368, 17713...|
1606596|762757| -1|2011-11-02 10:23:06| 1977| 1| 62|[28170, 8484, 628...| 1.4.2.4|[6359, 12152, 618...|
1606902|762757| -1|2011-11-02 10:23:06| 1977| 1| 62|[28170, 8484, 628...| 1.1.2.1|[619870, 242039, ...|
1837210|762847| -1|2011-10-31 09:35:11| 2009| 1| 0| []| 1.4.2.4|[97714, 174, 4877...|
+-------+------+------+-------------------+-------------+------+----------------+--------------------+--------+--------------------+
only showing top 20 rows

CPU times: user 40.6 ms, sys: 23.4 ms, total: 63.9 ms
Wall time: 16 s

#### Write the dataframe into Mongo DB

In [0]:
database = 'tencent'
collection = 'recologdetail'
user_name = '***'
password = '***'
address = '**'
connection_string = f"mongodb+srv://{user_name}:{password}@{address}/{database}.{collection}"

In [0]:
%%time
log_details.write.format("mongo").option("uri",connection_string).mode("overwrite").save()

CPU times: user 5.24 s, sys: 2.55 s, total: 7.79 s
Wall time: 33min 22s

In [0]:
spark.stop()